# README

---

VCP SDKを用いてクラウド上にOpenHPC環境を構築します。

## 概要

[OpenHPC](https://openhpc.community/)で配布しているパッケージを利用してクラウド上にHPC環境を構築します。

### 構成について

構築するOpenHPC環境は１つのマスターノード（ヘッドノード）と複数の計算ノードによって構成されます。
マスターノードはNFSサーバとしての役割も担います。NFSによってホームディレクトリやライブラリなどのファイルを計算ノードとの間で共有します。

OpenHPCではジョブスケジューラとして[Slurm](https://www.schedmd.com/)と[OpenPBS](https://www.openpbs.org/)が選択できますが、このテンプレートではSlurmを使用します。

![構成](images/ohpc-000.png)

### 構築方法について

OpenHPCのInstall guideでは[Warewulf](http://warewulf.lbl.gov/)などを用いて計算ノードのプロビジョニングを行っています。このテンプレートでは Warewulf などを利用せずに、VCP SDKを用いてマスターノードと計算ノードのプロビジョニングを行います。VCP SDKを用いることでクラウド上に仮想サーバを作成することができます。またNFS用の仮想ディスクもVCP SDKによって作成します。VCP SDKが作成する仮想サーバ、仮想ディスクのことを VCノード、VCディスクと呼びます。

#### VCノードの作成

VCノード（クラウド上の仮想サーバ）を作成するには、このNotebook環境に用意されているVCP SDKを利用します。VCP SDKはNotebook環境から利用できるPython3のライブラリで、パブリッククラウド上(AWS, Azure, ...)に仮想サーバや仮想ディスクを作成する操作を簡単に行うことができます。

実際のクラウドに対する操作はVCP SDKが直接行うのではなくVC Controllerと呼ばれるVCPのサーバによって実行されます。VC ControllerのAPIをNotebook環境から直接利用することも可能ですが、利用しやすいようにライブラリとしてまとめたものが VCP SDKになります。VC Contorollerを利用するには、アクセストークンが必要となります。VCP SDKを利用する場合も同様にVC Controllerのアクセストークンが必要となりますので、事前にVC管理者に対してアクセストークンの発行を依頼しておいてください。

#### VCノードのイメージ選択

VCPでは利用者に対して、異なるクラウド(AWS, Azure, ...)の仮想サーバを同一の環境として提供するために[Docker コンテナ](https://www.docker.com/)を利用しています。クラウドのVM/BMをVCノードとして組み入れる際にVCPが各VM/BMの上で共通環境となるコンテナを起動します。VCPではこのコンテナのことをBaseコンテナと呼びます。Baseコンテナは仮想サーバのモニタリングなどのVCノードに共通する機能を提供する役割も担っています。

標準のBaseコンテナは共通環境を提供することを目的としていますが、それとは別に特定の利用目的を持ったBaseコンテナを
アプリケーションテンプレートでは提供しています。OpenHPC向けには、マスターノード用Baseコンテナイメージと計算ノード用Baseコンテナイメージを用意しています。

#### VCディスクの作成

VCディスク（クラウド上の仮想ディスク）もVCノードと同様にVCP SDKを利用することで作成できます。

現在VCディスクを作成できるプロバイダは `AWS` ,`Azure` , `Oracle Cloud` に限定されています。その他のプロバイダでOpenHPC環境を構築することを考慮して、このアプリケーションテンプレートではマスターノードにNFS用ディスクを作成しない構成にも対応しています。NFS用ディスクを作成しない場合は仮想ノードのルートボリュームを直接利用するので、ルートボリュームサイズに大きめの値を指定してください。

### バージョン

このテンプレートで構築するミドルウェア、OSのバージョンを示します。

* [OpenHPC 2.4](https://github.com/openhpc/ohpc/releases/tag/v2.4.GA)
* Rocky Linux 8.5

### 前提条件

#### 時刻同期

OpenHPCでは各ノードの時刻が正しく設定されている必要があります。

VCPで時刻同期を行うには、リリースノート「[Release/20.10.0 -- 2.機能追加](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=32677360#id-%E3%83%AA%E3%83%AA%E3%83%BC%E3%82%B9%E3%83%8E%E3%83%BC%E3%83%88-Release/20.10.0(2020/10/30))」に記されているようにVCコントローラへの設定が必要となります。
OpenHPCの構築を行う前にOCS運用担当者にVCコントローラへの設定を依頼してください。

## Notebookについて

OpenHPCテンプレートのNotebookについて記します。

### 実行方法

テンプレートのNotebookを実行するには、下記に示すいずれかの手順で作業用Notebookを作成する必要があります。

* [000-README.ipynb#作業用notebookの作成](000-README.ipynb#作業用Notebookの作成)で作業用のNotebookを作成する
* テンプレートのNotebookを配置してあるディレクトリ [`notebooks/`](./notebooks/) から、`000-README.ipynb`(このNotebook)と同じディレクトリに手作業でコピーする

***注意:***

 [`notebooks/`](./notebooks/)に配置してあるNotebokをそのまま実行すると、他のファイルのパスなどが想定しているものと異なるため、
***正しく処理できずエラーとなります***。

### 実行順序

テンプレートのNotebookを実行する順序について記します。

はじめに全てのNotebookの関係について示し、その後に具体的な目的に応じた実行手順を例示します。
具体例としては以下のものを示します。

* LINPACKを実行する
* NGC(NVIDIA GPU CLOUD)カタログのコンテナを実行する

#### Notebook間の関係について

次のセルを実行すると、テンプレートにあるNotebook間の関連を示す図を表示します。ひとつの長方形の枠がそれぞれテンプレートのNotebookに対応しています。

> 図が表示されずに `<IPython.core.display.SVG object>` と表示されている場合は、次のセルを `unfreeze` した後に再実行してください。

In [ ]:
from IPython.display import SVG
%run scripts/nb_utils.py
setup_diag()
SVG(filename=generate_svg_diag(diag='images/notebooks.diag'))

#### LINPACKを実行する

OpenHPC環境で[HPL(High-Performance Linpack Benchmark)](http://www.netlib.org/benchmark/hpl/)を実行する手順の例を示します。次のセルを実行すると実行順序の図が表示されます。

In [ ]:
SVG(filename=generate_svg_diag(diag='images/notebooks-linpack.diag'))

まずは「環境構築」に表示されている、以下の４つのNotebookでOpenHPC環境の構築を行います。

* 010: パラメータ設定
* 020: OpenHPCの起動
* 031: 設定ファイルの編集 - slurm.conf
* 051: ユーザの追加

その後、構築したOpenHPC環境でHPLをジョブとして実行します。ジョブの実行は構築の際に利用した管理権限のあるユーザではなく「051:ユーザの追加」で作成した一般ユーザによって行うことを想定しています。

* 121: Linpackベンチマーク - HPL

#### NGC(NVIDIA GPU CLOUD)カタログのコンテナをSingularityで実行する

OpenHPC環境で[NGCカタログ](https://ngc.nvidia.com/)のコンテナをSingularityで実行する手順の例を示します。次のセルを実行すると実行順序の図が表示されます。

In [ ]:
SVG(filename=generate_svg_diag(diag='images/notebooks-gpu.diag'))

まずは「環境構築」に表示されている５つのNotebookでOpenHPC環境の構築を行います。ここではGPUを利用したジョブを実行することを想定してます。そのため「032: 設定ファイルの編集 - GPU(GRES)の登録」により Slurm にGPUをリソース登録します。またコンテナを実行するために「061: Singularityのロード」を用いてSingularityのセットアップを行います。

* 010: パラメータ設定
* 020: OpenHPCの起動
* 032: 設定ファイルの編集 - GPU(GRES)の登録
* 061: Singularityのロード
* 051: ユーザの追加

その後、構築したOpenHPC環境でNGCカタログのコンテナを利用したジョブを実行します。ジョブの実行は構築の際に利用した管理権限のあるユーザではなく「051:ユーザの追加」で作成した一般ユーザによって行うことを想定しています。この実行例ではNGCカタログのコンテナを実行する前に「140: Singularityの利用」によってSingularityを用いたコンテナ実行の手順を確認しています。

* 140: Singularityの利用
* 141: NGC Catalogのコンテナを実行する - PyTorch-Singularity
* 142: NGC Catalogのコンテナを実行する - TensorFlow-Singularity

***注意:***

NGCカタログのコンテナを実行する場合は、環境構築で実行する「010: パラメータ設定」のNotebookで以下のような設定が必要となります。

* 4.1.3 計算ノードのインスタンスタイプ
    - `compute_instance_type`にNVIDIAのGPUを利用できるインスタンスタイプ、VMサイズを指定してください
    - 動作確認済のインスタンスタイプ・VMサイズを以下に示します
        + AWS
            - [G4(NVIDIA T4)](https://aws.amazon.com/jp/ec2/instance-types/g4/)
            - [G3(NVIDIA M60)](https://aws.amazon.com/jp/ec2/instance-types/g3/)
            - [P3(NVIDIA V100)](https://aws.amazon.com/jp/ec2/instance-types/p3/)
        + Azure
            - [NC T4_v3(NVIDIA T4)](https://docs.microsoft.com/ja-jp/azure/virtual-machines/nct4-v3-series)
            - [NCv3(NVIDIA V100)](https://docs.microsoft.com/ja-jp/azure/virtual-machines/ncv3-series)
            - [NV(NVIDIA M60)](https://docs.microsoft.com/ja-jp/azure/virtual-machines/nv-series)
            - [NVv3(NVIDIA M60)](https://docs.microsoft.com/ja-jp/azure/virtual-machines/nvv3-series)
        + Oracle Cloud
            - [VM.GPU2(NVIDIA P100)](https://www.oracle.com/jp/cloud/compute/gpu.html)
    - AWS P2などのNVIDIA K80ではCUDA Compute Capabilityが3.7なのでNGCカタログのコンテナが実行できません

* 4.1.5 計算ノードにおけるGPUの利用
    - `compute_use_gpu`の値に`True` を指定してください
* 4.2.3 マスターノードのルートボリュームサイズ
    - コンテナイメージをSingularityで利用できるように処理するための作業領域としてある程度以上のサイズが必要となります
    - NGCカタログのPyTorchまたはTensorFlowのコンテナイメージを利用するには、`master_root_size`に少なくとも 60GB 以上の値を指定してください

> 上に示した「4.1.3」などの表記は Notebook「010: パラメータ設定」における章番号になります。

#### NGC(NVIDIA GPU CLOUD)カタログのコンテナをDockerで実行する

[NGCカタログ](https://ngc.nvidia.com/)のコンテナをDockerで実行する手順の例を示します。次のセルを実行すると実行順序の図が表示されます。前節との違いはコンテナの実行に SingularityではなくDockerを利用することになります。

In [ ]:
SVG(filename=generate_svg_diag(diag='images/notebooks-gpu-docker.diag'))

まずは「環境構築」に表示されている５つのNotebookでOpenHPC環境の構築を行います。ここではGPUを利用したジョブを実行することを想定してます。そのため「032: 設定ファイルの編集 - GPU(GRES)の登録」により Slurm にGPUをリソース登録します。またユーザにdockerコマンドの実行を許可するために「051: ユーザの追加」で追加するユーザに `docker` グループへの追加が必要となります。

* 010: パラメータ設定
* 020: OpenHPCの起動
* 032: 設定ファイルの編集 - GPU(GRES)の登録
* 071: Docker Engineのインストール
* 051: ユーザの追加

その後、構築したOpenHPC環境でNGCカタログのコンテナを利用したジョブを実行します。

* 151: NGC Catalogのコンテナを実行する - PyTorch-Docker
* 152: NGC Catalogのコンテナを実行する - TensorFlow-Docker

### 各Notebookの目次

次のセルを実行すると、テンプレートの全てのNotebookの目次が表示されます。

> 目次が表示されずに `<IPython.core.display.Markdown object>` と表示されている場合は、次のセルを `unfreeze` した後に再実行してください。

リンクが表示されている項目が一つのNotebookに対応しており、そのサブ項目が各Notebook内の目次になっています。

In [ ]:
from IPython.display import Markdown
%run scripts/nb_utils.py
Markdown(notebooks_toc())

## 作業用Notebookの作成

この節のセルを実行することで、テンプレートのNotebookから作業用Notebookを作成することができます。

まず、作業用Notebookを配置するディレクトリを指定してください。

In [ ]:
WORK_DIR = 'work'

次のセルを実行すると、Notebook名のドロップダウンリストと「作業開始」ボタンが現れます。
「作業開始」ボタンを押すと、テンプレートのNotebookからコピーを作成した後、自動的にブラウザでコピーが開きます。
Notebookの説明を確認しながら実行、適宜修正しながら実行していってください。

> このNotebookを Shutdown した後に再度開いた場合、次のセルに既に表示されている「作用開始」ボタンが正しく動作しません。次のセルをいったん unfreeze した後、セルを再実行してから「作業開始」ボタンをクリックして下さい。

In [ ]:
from IPython.core.display import HTML
%run scripts/nb_utils.py
setup_nb_workdir(WORK_DIR)
HTML(generate_html_work_nbs(WORK_DIR))